In [57]:
import pickle as pkl
import pandas as pd
import math

In [58]:
def check_pkl_headers(file_path):
    try:
        # Open the .pkl file
        with open(file_path, "rb") as file:
            data = pkl.load(file)

        # Check if the data is a dictionary
        if isinstance(data, dict):
            print("Keys in the dictionary (headers):")
            print(data.keys())  # Print the keys of the dictionary

            # Print the first value in each header (key)
            print("\nFirst value for each header:")
            for key, value in data.items():
                if isinstance(value, list) or isinstance(
                    value, tuple
                ):  # Check if it's a list or tuple
                    print(
                        f"{key}: {value[0]}"
                    )  # Print the first value of the list or tuple
                else:
                    print(
                        f"{key}: {value}"
                    )  # Print the value (if not list/tuple, print as it is)
        else:
            print("The data is not a dictionary.")
            print("Data type:", type(data))
            print("Content:", data)

    except Exception as e:
        print(f"Error loading the file: {e}")

In [59]:
file_path = "Process\FTG_data_test_1.pkl"
check_pkl_headers(file_path)

Keys in the dictionary (headers):
dict_keys(['observations', 'actions', 'terminals', 'rewards'])

First value for each header:
observations: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5440000295639038, 0.550000011920929, 0.5339999794960022, 0.5350000262260437, 0.5450000166893005, 0.5479999780654907, 0.5490000247955322, 0.5580000281333923, 0.5529999732971191, 0.5379999876022339, 0.5370000004768372, 0.5490000247955322, 0.5580000281333923, 0.5559999942779541, 0.5389999747276306, 0.5479999780654907, 0.5609999895095825, 0.5680000185966492, 0.5699999928474426, 0.5569999814033508, 0.5669999718666077, 0.5440000295639038, 0.5460000038146973, 0.5529999732971191, 0.5529999732971191, 0.5590000152587891, 0.5630000233650208, 0.5550000071525574, 0.5640000104904175, 0.5509999990463257, 0.5600000023841858, 0.5640000104904175, 0.5580000281333923, 0.5659999847412109, 0.5519999861717224, 0.5600000023841858, 0.5600000023841858, 0.5659999847412109, 0.5

In [60]:
def remove_zeros_and_find_min(arr):
    # Remove 0s from the array
    arr_no_zeros = [x for x in arr if x != 0]

    # Check if the array is empty after removing zeros
    if not arr_no_zeros:
        return 1  # Return None if no elements are left

    # Find the minimum value in the modified array
    min_value = min(arr_no_zeros)

    return min_value

In [61]:
def process_side(data):
    #clean the data by removing 0s and finding the minimum value used to calculate reward
    minimum = remove_zeros_and_find_min(data)
    if minimum >0.35:
        return 0
    elif minimum > 0.19:
        reward = (-1/(5.7*minimum-1))
    else:
        #crash
        return -50
    return reward/2

In [62]:
def process_front(data):
    # clean the data by removing 0s and finding the minimum value used to calculate reward
    minimum = remove_zeros_and_find_min(data)
    if minimum > 0.4:
        return 0
    elif minimum > 0.19:
        reward = -1 / (5.7 * minimum - 1)
    else:
        # crash
        return -50
    return reward/2


In [63]:
def calculate_reward(observation, action):
    # Implement logic to calculate the reward based on the observation and action
    reward = 0.0  # Initalize the reward
    # action space
    speed, steering_angle = action  # split the action space
    reward += speed**2 - 1.5  # calaulate reward based on speed
    reward += math.exp(-(steering_angle**2))  # calculate reward based on steering angle

    # observation space
    # Lidar goes right to left
    # sections (Left, Right), (Front Left, Front Right)
    right = observation[0:360]
    front_right = observation[360:540]
    front_left = observation[540:720]
    left = observation[720:]
    reward += process_side(left)
    reward += process_side(right)
    reward += process_front(front_left)
    reward += process_front(front_right)

    return reward

In [64]:
def non_zero_minima(lst):
    # Filter out zero values
    non_zero_lst = [x for x in lst if x != 0]

    # If there are no non-zero values, return None or an appropriate message
    if not non_zero_lst:
        return None

    # Return the minimum value from the non-zero list
    return min(non_zero_lst)

In [ ]:
def evaluate_trajectory(data):
    try:
        # Extract values from the dictionary
        observations = data.get("observations", [])
        actions = data.get("actions", [])
        terminals = data.get("terminals", [])
        rewards = data.get("rewards", [])

        # Ensure all lists have the same length
        if not (len(observations) == len(actions) == len(terminals) == len(rewards)):
            raise ValueError(
                "Mismatch in the length of observations, actions, terminals, and rewards"
            )
        else:
            print(f"The total number of State Action Transitions is: {len(observations)}")

        # Iterate through the observations and actions to evaluate rewards and terminal state
        total_reward = 0

        minima = 5
        for t in range(len(observations)):
            obs = observations[t]
            action = actions[t]
            terminal = terminals[t]

            # Placeholder: Calculate reward for the current observation and action
            reward = calculate_reward(obs, action)
            total_reward +=reward

            # Append the reward to the rewards list
            rewards[t] = reward

            # Print or log the state and reward information for debugging purposes

            
            x = non_zero_minima(obs)
            if x< minima:
                minima = x

            print(
                f"Step {t}: Observation: {x}, Action: {action}, Reward: {reward}, Terminal: {terminal}, Cumulative Reward: {total_reward}"
            )
            # if reward <=0:
            #     print(
            #     f"Step {t}: Observation: {obs}, Action: {action}, Reward: {reward}, Terminal: {terminal}, Cumulative Reward: {total_reward}"
            #     )
            # else:
            #     print(f"Step {t}")

            # Check if terminal state has been reached
            if terminal:
                print(f"Terminal state reached at step {t}. Ending trajectory.")
                break  # Or reset trajectory if necessary


        # Optionally return the modified rewards list
        return rewards

    except Exception as e:
        print(f"Error in evaluating trajectory: {e}")

In [75]:
file_path = "Process\FTG_data_test_1.pkl"
with open(file_path, "rb") as file:
    data = pkl.load(file)

if isinstance(data, dict):
    print("Starting trajectory:")
    evaluate_trajectory(data)
else:
    print("File Failed!")

Starting trajectory:
The total number of State Action Transitions is: 4800
Step 0: Observation: 0.5339999794960022, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 6.669181399411159
Step 1: Observation: 0.5299999713897705, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 13.338362798822319
Step 2: Observation: 0.5389999747276306, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 20.00754419823348
Step 3: Observation: 0.5460000038146973, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 26.676725597644637
Step 4: Observation: 0.5410000085830688, Action: (2.706497673953753, -0.061086579225957394), Reward: 6.821405042611441, Terminal: False, Cumulative Reward: 33.49813064025608
Step 5: Observation: 0.5299999713897705, Action: (2.7

In [76]:
def evaluate_trajectory_dic(data):
    try:
        # Extract values from the dictionary
        observations = data.get("observations", [])
        actions = data.get("actions", [])
        terminals = data.get("terminals", [])
        rewards = data.get("rewards", [])

        # Ensure all lists have the same length
        if not (len(observations) == len(actions) == len(terminals) == len(rewards)):
            raise ValueError(
                "Mismatch in the length of observations, actions, terminals, and rewards"
            )
        else:
            print(
                f"The total number of State Action Transitions is: {len(observations)}"
            )

        # Iterate through the observations and actions to evaluate rewards and terminal state
        total_reward = 0
        minima = 5
        for t in range(len(observations)):
            obs = observations[t]
            action = actions[t]
            terminal = terminals[t]

            # Placeholder: Calculate reward for the current observation and action
            reward = calculate_reward(obs, action)
            total_reward += reward

            # Append the reward to the rewards list
            rewards[t] = reward

            # Print or log the state and reward information for debugging purposes
            x = non_zero_minima(obs)
            if x < minima:
                minima = x

            print(
                f"Step {t}: Observation: {x}, Action: {action}, Reward: {reward}, Terminal: {terminal}, Cumulative Reward: {total_reward}"
            )

            # Check if terminal state has been reached
            if terminal:
                print(f"Terminal state reached at step {t}. Ending trajectory.")
                break  # Or reset trajectory if necessary

        # Create a new dictionary with the updated rewards and other relevant information
        result = {
            "observations": observations,
            "actions": actions,
            "terminals": terminals,
            "rewards": rewards,  # Updated rewards list
            "minima": minima,  # The minimum non-zero value observed
            "total_reward": total_reward,  # The cumulative total reward
        }

        # Return the dictionary containing the updated data
        return result

    except Exception as e:
        print(f"Error in evaluating trajectory: {e}")

In [77]:
import os
import pickle as pkl
import glob


def process_and_save_pkl_files(input_folder, output_folder):
    try:
        # Ensure the output folder exists, create it if it doesn't
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Get all .pkl files in the input folder using glob
        pkl_files = glob.glob(os.path.join(input_folder, "*.pkl"))

        # Check if no .pkl files are found
        if not pkl_files:
            print("No .pkl files found in the input folder.")
            return

        # Iterate through each .pkl file in the folder
        for file_path in pkl_files:
            print(f"Processing file: {file_path}")

            # Open and load the .pkl file
            with open(file_path, "rb") as file:
                data = pkl.load(file)

            # Check if the data is a dictionary, indicating it's a trajectory
            if isinstance(data, dict):
                print("Starting trajectory:")
                result = evaluate_trajectory_dic(
                    data
                )  # Call your evaluate_trajectory function here
                print(f"Processed trajectory result: {result}")

                # Define the output file path (same name as input file, but in the output folder)
                output_file_path = os.path.join(
                    output_folder, os.path.basename(file_path)
                )

                # Save the processed result (the dictionary) to a .pkl file in the output folder
                with open(output_file_path, "wb") as output_file:
                    pkl.dump(result, output_file)
                print(f"Saved processed result to: {output_file_path}")

            else:
                print(f"File {file_path} is not a valid trajectory or data format!")

    except Exception as e:
        print(f"Error processing files in folder: {e}")

In [ ]:
process_and_save_pkl_files("Process", "Data")

Processing file: Process\FTG_data_test_1.pkl
Starting trajectory:
The total number of State Action Transitions is: 4800
Step 0: Observation: 0.5339999794960022, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 6.669181399411159
Step 1: Observation: 0.5299999713897705, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 13.338362798822319
Step 2: Observation: 0.5389999747276306, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 20.00754419823348
Step 3: Observation: 0.5460000038146973, Action: (2.6782786018066673, -0.06326824077405035), Reward: 6.669181399411159, Terminal: False, Cumulative Reward: 26.676725597644637
Step 4: Observation: 0.5410000085830688, Action: (2.706497673953753, -0.061086579225957394), Reward: 6.821405042611441, Terminal: False, Cumulative Reward: 33.49813064025608
Step 5: 